## Imports

In [83]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Point

## Read-in

In [84]:
df = pd.read_csv('LISF_Dec_2023.csv',dtype='str')

In [85]:
# Make sure to update this
first_month = 12
current_month = 'December'

In [86]:
len(df)

923

In [87]:
# Define the regex pattern to split the text
pattern = r'(\d+-\d+-\d+-\d+-\d+)\s(.*)'

# Apply regex and split the text into two columns
df[['PIN', 'Address']] = df['1st PIN'].str.extract(pattern)

# Remove leading/trailing whitespace from the address column
df['Address'] = df['Address'].str.strip()

In [88]:
df = df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0','1st PIN'])

## Clean, drop, and convert

In [89]:
df['Doc Recorded'] = pd.to_datetime(df['Doc Recorded'], format='mixed')

In [90]:
df = df.drop_duplicates()

## Data Stuff

In [10]:
# # Count number of earliest month dates
# sep_dates_count = len(df.loc[df['Doc Recorded'].dt.month == first_month])

# print(f'Number of September dates: {sep_dates_count}')

In [91]:
df['1st Grantor'] = df['1st Grantor'].fillna('NA')
df['1st Grantee'] = df['1st Grantee'].fillna('NA')

In [92]:
municipal_authority = ['city', 'town', 'municipality', 'village','transit auth','department of transp']  # list of municipal authority keywords
df = df[~df['1st Grantor'].str.contains('|'.join(municipal_authority), case=False) & 
        ~df['1st Grantee'].str.contains('|'.join(municipal_authority), case=False)]

In [93]:
# create new column and assign colors based on month
df['COLOR'] = df['Doc Recorded'].apply(lambda x: 'red' if x.month == first_month else '')

## Get mortgage amounts

In [94]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'
}

In [95]:
def mortgage_url_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    for link in soup.find_all('a', href=True):
        if link['href'].startswith('/Document/Detail'):
            mortgage_url = 'https://crs.cookcountyclerkil.gov' + link['href']
            return mortgage_url

In [96]:
def mortgage_consi_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find all tr tags
    trs = soup.find_all('tr')

    # Loop through each tr tag and look for the td tag containing 'Consideration Amount' label
    for tr in trs:
        td = tr.find('th', text='Consideration Amount:')
        if td:
            # If the td tag is found, get the next td tag containing the amount
            amount_td = td.find_next_sibling('td')
            if amount_td:
                # Print the amount
                amount = amount_td.text.strip()
                return amount
            else:
                return 'not found'

In [97]:
df['mortgage_urls'] = df['deed_urls'].apply(lambda x: mortgage_url_snagger(x, headers))

In [98]:
df['mortgage_amount'] = df['mortgage_urls'].apply(lambda x: mortgage_consi_snagger(x, headers) if x is not None else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_10736/2390556658.py:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  td = tr.find('th', text='Consideration Amount:')


In [99]:
df[['mortgage_urls','mortgage_amount']] = df[['mortgage_urls','mortgage_amount']].fillna('NA')

## Geocode

In [100]:
df['geo_address'] = df['Address'] + ' Cook County, IL'

In [101]:
len(df)

612

In [102]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [103]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['geo_address'].apply(geocode)

In [104]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## HTML Popup Formatter

In [105]:
df.columns

Index(['View Doc', 'Doc Number', 'Doc Recorded', 'Doc Executed', 'Doc Type',
       'Consi. Amt.', '1st Grantor', '1st Grantee', 'Assoc. Doc#', 'deed_urls',
       'PIN', 'Address', 'COLOR', 'mortgage_urls', 'mortgage_amount',
       'geo_address', 'geocoded', 'lat', 'lon', 'mortgage_amount_int'],
      dtype='object')

In [106]:
def popup_html(row):
    grantor = row['1st Grantor']
    grantee = row['1st Grantee']
    PIN = row['PIN']
    Address = row['Address']
    mortgage_amount = row['mortgage_amount']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Lender: </strong>{}'''.format(grantor) + '''<br>
    <strong>Borrower: </strong>{}'''.format(grantee) + '''<br>
    <strong>PIN: </strong>{}'''.format(PIN) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Mortgage Amount: </strong>{}'''.format(mortgage_amount) + '''<br>
    </html>
    '''
    return html

In [107]:
import folium
from folium.plugins import MarkerCluster
import numpy as np

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Cook County Pending Foreclosures')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{current_month}')

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9.5, tiles=None)

# Create two FeatureGroups for different color pins
fg_red = folium.FeatureGroup(name=f'{current_month}') ## 

for index, row in df.iterrows():
    lat = row['lat']
    lon = row['lon']
    color = row['COLOR']
    if pd.notnull(lat) and pd.notnull(lon) and color == 'red':
        marker = folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            fill=True,
            color=color,
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_red)
    else:
        continue

# Add the FeatureGroups to the map
fg_red.add_to(m)

folium.TileLayer('OpenStreetMap', control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
folium.TileLayer('CartoDBpositron', control=False).add_to(m) 
            
# Display map
m

In [108]:
m.save('index.html')

## Map URL Snagger

In [109]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/lis_pendens_scraper_december_2023


## Stats for story

In [110]:
df['mortgage_amount_int'] = df['mortgage_amount'].str.replace('$','')
df['mortgage_amount_int'] = df['mortgage_amount_int'].str.replace(',','')
df = df.loc[df['mortgage_amount_int'] != 'NA']
df['mortgage_amount_int'] = df['mortgage_amount_int'].astype(float).fillna(0).astype(int)

In [111]:
# Count number of first month dates
dates_count = len(df.loc[df['Doc Recorded'].dt.month == first_month])

print(f'Number of {current_month} dates: {dates_count}')

Number of December dates: 265


In [126]:
# df.loc[df['Doc Recorded'].dt.month == first_month]

In [113]:
x = df.loc[df['Doc Recorded'].dt.month == first_month]

In [114]:
x.reset_index(inplace=True, drop=True)

In [36]:
# x.at[123,'mortgage_amount_int']=33130000
# x.at[123,'mortgage_amount']='$33,130,000'

# x.at[16,'mortgage_amount_int']=173600
# x.at[16,'mortgage_amount']='$173,600'

In [127]:
x.sort_values(by='mortgage_amount_int',ascending=False).iloc[0:1]

,View Doc,Doc Number,Doc Recorded,Doc Executed,Doc Type,Consi. Amt.,1st Grantor,1st Grantee,Assoc. Doc#,deed_urls,PIN,Address,COLOR,mortgage_urls,mortgage_amount,geo_address,geocoded,lat,lon,mortgage_amount_int
200,View,2334606112,2023-12-12,12/8/2023,LIS PENDENS FORECLOSURE,NaN,COMM 2014-UBS2 NATL STATE STREET LLC,TBG STATE ST LLC,1404926039.0,https://crs.cookcountyclerkil.gov/Document/Det...,17-10-311-051-0000,"30 E MADISON ST, CHICAGO",red,https://crs.cookcountyclerkil.gov/Document/Det...,"$60,000,000.00","30 E MADISON ST, CHICAGO Cook County, IL","(41.8821908, -87.6265945)",41.882191,-87.626594,60000000


In [116]:
print(x['mortgage_urls'].iloc[110])

https://crs.cookcountyclerkil.gov/Document/Detail?dId=Mjc0MDE3NzE1&hId=ZDI1MDdmYjBkYmQwNjQ4N2ZiNTQ2MDUwOGRjMGQ3ZWYxOGZlZWZkZmRiNzhhNTRiNDQ5MDdjYzcyM2NiNzNhMA2


In [117]:
print(f'{current_month}: ${x.mortgage_amount_int.sum():,}')

December: $143,544,018


In [118]:
# October
16 + 12 + 5

33

In [119]:
x['1st Grantor'].value_counts().head(60)

1st Grantor
NEWREZ LLC                                25
WILMINGTON SAV FUND SOCIETY FSB TR        14
PNC BK NATIONAL ASSOCIATION               11
WELLS FARGO BK NA                         11
US BANK TRUST NATIONAL ASSN TR            10
LOANDEPOTCOM LLC                          10
LAKEVIEW LOAN SERVICING LLC                9
US BANK NATIONAL ASSN TR                   8
HUNTINGTON NATL BANK                       7
US BANK TRUST COMPANY NATIONAL ASSN TR     5
ASSOCIATED BK NA                           5
FIFTH THIRD BK NATIONAL ASSOCIATION        5
BMO BK NA                                  4
CARRINGTON MTG SERVICES LLC                4
TRUIST BANK                                3
COMMUNITY LOAN SERVICING LLC               3
SPECIALIZED LOAN SERVICING LLC             3
CITIZENS BK NA                             3
PHH MTG CORPORATION                        3
F ST INVESTMENTS LLC                       3
JPMORGAN CHASE BK NATIONAL ASSOCIATION     3
BRENDAN MTG INC                            

In [120]:
x['1st Grantee'].value_counts()

1st Grantee
SAWYERS AND MCDANIEL LLC            3
ST DANE HOMES LLC                   2
BLACK BOND HOLDINGS LLC             2
CHICAGO TITLE LAND TRUST CO TR      2
FAVELA FILIBERTO                    2
                                   ..
TRAVIS CYNTHIA T                    1
RAGANO MICHELLE                     1
DIESING BRAD                        1
DAVIS PATRICIA A                    1
STANDARD BK AND TRUST COMPANY TR    1
Name: count, Length: 255, dtype: int64

In [121]:
df.to_csv(f'{current_month}_2023_foreclosures.csv')

In [44]:
# apr.sort_values(by='mortgage_amount_int',ascending=False)

In [122]:
print(f"{current_month} median mortgage foreclosure amount: ${x['mortgage_amount_int'].median():,}")

December median mortgage foreclosure amount: $170,100.0


## Boundary Analysis

In [123]:
boundaries = gpd.read_file('Boundaries - Neighborhoods.geojson')

In [124]:
# create Point objects from lat/lon columns in df
geometry = gpd.points_from_xy(x['lon'], x['lat'])

# create geodataframe from df with Point objects as geometry
gdf_filings_x = gpd.GeoDataFrame(x, geometry=geometry)

gdf_filings_x.crs = 'EPSG:4326'
gdf_filings_x = gdf_filings_x.to_crs(boundaries.crs)

# use contains method to get count of points within each neighborhood
counts_x = gpd.sjoin(gdf_filings_x, boundaries, predicate='within').groupby('pri_neigh').size().reset_index(name='count')

In [125]:
counts_x['count'].sum()

153